## Device check

In [1]:
!nvidia-smi

Thu Oct 26 02:41:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:B2:00.0 Off |                    0 |
| N/A   30C    P0    39W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Packages

In [2]:
import warnings
warnings.filterwarnings(action="ignore")

In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

import lightning.pytorch as pl
import logging
logging.basicConfig(level="INFO")

import math
from matplotlib import pyplot

import numpy as np

import os

import pandas as pd
from pprint import pprint

import sys
sys.path.append("../src")

import timm
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
%load_ext autoreload
%autoreload 2

In [5]:
from einops import pack, unpack, repeat, reduce, rearrange

In [71]:
import config
from dataloader import BEDataset, BEDataModule
import token_learner
from transformer import PositionalEncoder, MultiHeadSelfAttention, FeedFowardLayer, generate_masks, generate_causal_attention_mask

from film_layers import FiLMBlockV2, FiLMEncoder, ResBlockDWConv
from rt1 import RT1Encoder
from utils.data_utils import History
from utils.model_utils import TextEncoder, ImageFeatureExtractor, plot_self_attention


## Data Module

In [ ]:
# csv = pd.read_csv(os.path.join(config.DATASET_PATH, "train.csv"))

# csv.head()

In [8]:
dm = BEDataModule()
dm.setup()

Total # examples: 4876


INFO:root:Training on 3915 samples.
INFO:root:Validating on 961 samples.


In [ ]:
# print("="*100)
# logging.info("\n>> train data loader")
# print(f"# train batches\t: {len(dm.train_dataloader())}")
# for data in dm.train_dataloader():
#     # pprint(data)
#     sample_id, in_state, ad, cmd = data["sample_id"], data["in_state"], data["action_desc"], data["motor_cmd"]
#     print("In \t\t\t: ", in_state.shape)
#     print("Action desc \t\t: ", ad["ids"].shape)
#     print("Action desc (len) \t: ", ad["length"].shape)
#     print("CMD \t\t\t: ", cmd["ids"].shape)
#     print("CMD(len) \t\t: ", cmd["length"].shape)
#     break

# print("\nIDs & decided tokens")
# for data in dm.train_dataloader():
#     print(data["action_desc"]["ids"][0].tolist())
#     print(dm.train_ds._decode_inputs(data["action_desc"]["ids"][0].tolist()))
#     print()
#     print(data["motor_cmd"]["ids"][0].tolist())
#     print(dm.train_ds._decode_outputs(data["motor_cmd"]["ids"][0].tolist()))

#     break
    
# print("="*100)

## Fetch batch

In [9]:
%%time
sample = next(iter(dm.train_dataloader()))
sample["in_state"].shape

CPU times: user 1.04 s, sys: 996 ms, total: 2.04 s
Wall time: 4.8 s


torch.Size([4, 3, 288, 288])

## Model Design

<!-- ![RT1 model architecture](../../imgs/rt1+.png) -->
<center>
    <img src="../imgs/rt1+.png" alt="RT1 model architecture" width="300" height="400">

<center>

### Encoder

#### Test Text Encoder

In [27]:
# te = TextEncoder(freeze=True).cuda()
# summary(model=te, col_names=["num_params", "trainable"])

In [13]:
# emb = te(
#     inp_ids=sample["action_desc"]["ids"].cuda(),
#     mask=sample["action_desc"]["mask"].cuda(),
#     tok_type_ids=sample["action_desc"]["token_type_ids"].cuda()
# )

# emb.shape

#### Test Img Feature Extractor

In [ ]:
# fe = ImageFeatureExtractor(pretrained=True, arch="efficientnet_b3").cuda()

# summary(fe, col_names=["num_params", "trainable"])

In [ ]:
# img_ftrs = fe(sample["in_state"].cuda())

# img_ftrs.shape

#### Test FiLM Block

In [16]:
# film_block = FiLMBlockV2().cuda()
# print(film_block)
# summary(model=film_block)

In [17]:
# text_cond_ftrs = film_block(
#     img_features=img_ftrs, 
#     conditioning=emb
# )

# text_cond_ftrs.shape

#### Test Residual FiLM Block

In [18]:
# dw_res = ResBlockDWConv(512, 512).cuda()
# summary(model=dw_res)

In [19]:
# text_cond_ftrs_res = dw_res(
#     img_features=img_ftrs, 
#     conditioning=emb
# )

# text_cond_ftrs_res.shape

#### Test FiLM Encoder

In [20]:
# film_encoder = FiLMEncoder(
#     arch="resnet34",
#     n_res_blocks=6,
# ).cuda()

# # print(film_encoder)
# summary(model=film_encoder)

In [21]:
# %%time

# out = film_encoder(
#     x= sample["in_state"].cuda(),
#     conditioning= emb
# )

# out.shape

#### Token Learner

In [22]:
# N, C, H_W = out.shape
# N, C, H_W

In [23]:
# tokL_v11 = token_learner.TokenLearnerModuleV11(feature_shape=(N, H_W, C)).cuda()
# print(tokL_v11)
# summary(model=tokL_v11)

In [24]:
# learned_tokens = tokL_v11(out.view(N, H_W, C))
# learned_tokens.shape

In [154]:
tokenized_inputs = torch.empty((B, config.NUM_HISTORY+1, config.D_MODEL, config.NUM_LEARNED_TOKENS))
tokenized_inputs.shape

torch.Size([8, 6, 512, 8])

#### RT-1 Encoder

In [35]:
encoder = RT1Encoder(
    cnn_bacnbone="efficientnet_b3", num_res_blocks=6, freeze_cnn_backbone=False
).to(config.DEVICE)

summary(model=encoder, col_names=["num_params", "trainable"])

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b3.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b3.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


Layer (type:depth-idx)                                       Param #                   Trainable
RT1Encoder                                                   --                        Partial
├─TextEncoder: 1-1                                           --                        False
│    └─BertModel: 2-1                                        --                        False
│    │    └─BertEmbeddings: 3-1                              (15,891,456)              False
│    │    └─BertEncoder: 3-2                                 (12,609,536)              False
│    │    └─BertPooler: 3-3                                  (262,656)                 False
│    └─Dropout: 2-2                                          --                        --
├─FiLMEncoder: 1-2                                           --                        True
│    └─ImageFeatureExtractor: 2-3                            --                        True
│    │    └─EfficientNetFeatures: 3-4                        10,103,3

In [36]:
%%time 

src_enc, tokens = encoder._encode(
    input_ids=sample["action_desc"]["ids"].cuda(),
    attn_mask=sample["action_desc"]["mask"].cuda(),
    token_type_ids=sample["action_desc"]["token_type_ids"].cuda(),
    imgs=sample["in_state"].cuda()
)

src_enc.shape, tokens.shape

CPU times: user 988 ms, sys: 1.13 s, total: 2.12 s
Wall time: 14.9 s


(torch.Size([4, 512]), torch.Size([4, 512, 8]))

In [37]:
%%time

src_enc, tokenized_inputs = encoder(
    input_ids=sample["action_desc"]["ids"].cuda(),
    attn_mask=sample["action_desc"]["mask"].cuda(),
    token_type_ids=sample["action_desc"]["token_type_ids"].cuda(),
    imgs=sample["in_state"].cuda()
)

tokenized_inputs.shape, tokenized_inputs.device

CPU times: user 263 ms, sys: 170 ms, total: 433 ms
Wall time: 2.48 s


(torch.Size([4, 48, 512]), device(type='cuda', index=0))

### Decoder

In [38]:
pos_encoding = PositionalEncoder(seq_len=config.NUM_TOKENIZED_INPUTS).cuda()

In [39]:
tokenized_inputs[0, 0][:3]

tensor([0.6933, 0.5674, 0.5251], device='cuda:0', grad_fn=<SliceBackward0>)

In [40]:
pos_enc = pos_encoding(tokenized_inputs)
print(pos_enc.shape)

learned_tokens = tokenized_inputs + pos_enc

torch.Size([4, 48, 512])


In [41]:
pos_enc[0, 0][:3]

tensor([15.6886, 13.8378, 11.8820], device='cuda:0', grad_fn=<SliceBackward0>)

In [42]:
learned_tokens[0, 0][:3]

tensor([16.3819, 14.4052, 12.4072], device='cuda:0', grad_fn=<SliceBackward0>)

In [43]:
attn_mask = generate_causal_attention_mask()
attn_mask.shape

torch.Size([48, 48])

#### Transformer Decoder

#### Cross Attention Layer

In [340]:
class CrossAttentionHead(nn.Module):
    def __init__(
        self, 
        embed_dim:int=config.D_MODEL, 
        dropout_rate:float=config.DROPOUT_RATE
    ):
        super().__init__()
        self.inf = 1e9
        self.embed_dim = embed_dim
        self.dropout_rate = dropout_rate
        
        # Linear transformations for queries, keys, and values for input sequences
        self.w_q_input = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_k_input = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_v_input = nn.Linear(embed_dim, embed_dim, bias=False)
        
        # Linear transformations for queries, keys, and values for output sequences
        self.w_q_output = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_k_output = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_v_output = nn.Linear(embed_dim, embed_dim, bias=False)

        self.dropout = nn.Dropout(dropout_rate)
        self._softmax = nn.Softmax(dim=-1)

    def forward(self, input_seq, output_seq):
        
        input_shape = input_seq.shape
        output_shape = output_seq.shape

        # Linear transformations + split into num_heads for both input and output sequences
        q_input = self.w_q_input(input_seq)
        k_input = self.w_k_input(input_seq)
        v_input = self.w_v_input(input_seq)
        
        q_output = self.w_q_output(output_seq)
        k_output = self.w_k_output(output_seq)
        v_output = self.w_v_output(output_seq)

        # Transpose dimensions for input and output sequences
        q_input = q_input.transpose(1, 2)
        k_input = k_input.transpose(1, 2)
        v_input = v_input.transpose(1, 2)
        q_output = q_output.transpose(1, 2)
        k_output = k_output.transpose(1, 2)
        v_output = v_output.transpose(1, 2)

        # Calculate cross-attention scores
        attn_scores = torch.matmul(q_output, k_input.transpose(-2, -1))  # (B, num_heads, L_output, L_input)
        attn_scores = attn_scores / math.sqrt(self.embed_dim)

        # Softmax
        attn_W = self._softmax(attn_scores)
        attn_W = self.dropout(attn_W)

        # Calculate cross-attention values
        attn_values = torch.matmul(attn_W, v_input)  # (B, num_heads, L_output, embed_dim)

        return attn_values, attn_W
        

class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout_rate):
        super(CrossAttention, self).__init__()
        
        self.inf = 1e9
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = self.embed_dim // self.num_heads
        
        # Linear transformations for queries, keys, and values for input and output sequences
        self.w_q_input = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_k_input = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_v_input = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_q_output = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_k_output = nn.Linear(embed_dim, embed_dim, bias=False)
        self.w_v_output = nn.Linear(embed_dim, embed_dim, bias=False)

        self.dropout = nn.Dropout(dropout_rate)
        self._softmax = nn.Softmax(dim=-1)

    def forward(self, input_seq, output_seq):
        input_shape = input_seq.shape
        output_shape = output_seq.shape

        # Linear transformations + split into num_heads for both input and output sequences
        q_input = self.w_q_input(input_seq).view(input_shape[0], -1, self.num_heads, self.d_k)
        k_input = self.w_k_input(input_seq).view(input_shape[0], -1, self.num_heads, self.d_k)
        v_input = self.w_v_input(input_seq).view(input_shape[0], -1, self.num_heads, self.d_k)
        q_output = self.w_q_output(output_seq).view(output_shape[0], -1, self.num_heads, self.d_k)
        k_output = self.w_k_output(output_seq).view(output_shape[0], -1, self.num_heads, self.d_k)
        v_output = self.w_v_output(output_seq).view(output_shape[0], -1, self.num_heads, self.d_k)

        # Transpose dimensions for input and output sequences
        q_input = q_input.transpose(1, 2)
        k_input = k_input.transpose(1, 2)
        v_input = v_input.transpose(1, 2)
        q_output = q_output.transpose(1, 2)
        k_output = k_output.transpose(1, 2)
        v_output = v_output.transpose(1, 2)

        # Calculate cross-attention scores
        attn_scores = torch.matmul(q_output, k_input.transpose(-2, -1))  # (B, num_heads, L_output, L_input)
        attn_scores = attn_scores / math.sqrt(self.embed_dim)

        # Softmax
        attn_W = self._softmax(attn_scores)
        attn_W = self.dropout(attn_W)

        # Calculate cross-attention values
        attn_values = torch.matmul(attn_W, v_input)  # (B, num_heads, L_output, embed_dim)

        return attn_values, attn_W

In [345]:
input_seq = torch.randn(3, 10, 512) 
output_seq = torch.randn(3, 12, 512)  

In [ ]:
num_heads = 4
cross_attention = CrossAttention(embed_dim=512, num_heads=num_heads, dropout_rate=0.1)

summary(cross_attention

In [346]:
cross_attention.eval()

attention_v, attention_w = cross_attention(input_seq, output_seq)
attention_v.shape, attention_w.shape

(torch.Size([3, 4, 12, 128]), torch.Size([3, 4, 12, 10]))

#### Plot Cross attention

In [347]:
fig, axes = pyplot.subplots(1, num_heads, figsize=(15, 5))
for head_idx in range(num_heads):
    attention_weights_example_head = attention_w[example_idx, head_idx].cpu().detach().numpy()
    ax = axes[head_idx]
    ax.imshow(attention_weights_example_head, cmap='viridis')
    ax.set_title(f'Head {head_idx + 1}')
pyplot.show()

In [278]:
class TransformerDecoderLayer(nn.Module):
    def __init__(
        self, 
        d_model:int=config.D_MODEL, 
        nhead:int=config.N_HEADS, 
        dim_feedforward:int=config.D_FF, 
        dropout:float=config.DECODER_DROPOUT_RATE
    ):
        super().__init__()

        # Multi-head self-attention
        self.self_attn = MultiHeadAttention()
        
        # Layer normalization 1
        self.norm1 = LayerNormalization()
        
        self.multihead_attn = MultiHeadAttention()

        # Layer normalization 2
        self.norm2 = LayerNormalization()
        
        # Position-wise feed-forward network
        self.ff = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.GELU(),
            nn.Linear(dim_feedforward, d_model)
        )
        
        # Layer normalization after feed-forward
        self.norm3 = nn.LayerNorm(d_model)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(
        self, 
        inp:torch.Tensor, 
        mask:torch.Tensor=None, 
        return_weights:bool=False
    ):
        
        # Layer normalization 1
        inp = self.norm1(inp)
        
        # Multi-head self-attention
        attn_W = None
        self_attn_output = self.self_attn(inp, inp, inp, mask=mask)
        if return_weights:
            self_attn_output, attn_W = self_attn_output
        
        # Apply dropout and add the residual connection
        inp = inp + self.dropout(self_attn_output)
        
        # Layer normalization 2
        out = self.norm2(inp)
        
        # Apply FF
        ff_out = self.dropout(self.ff(out))
        
        # Add the residual connection
        inp = inp + ff_out

        return inp, attn_W
    

class TransformerDecoder(nn.Module):
    def __init__(
        self, 
        num_layers:int=config.N_DECODER_LAYERS
    ):
        super().__init__()
        
        self.num_layers = num_layers
        # self.target_embedding = nn.Embedding(
        #     num_embeddings=config.TARGET_VOCAB_SIZE, 
        #     embedding_dim=config.D_MODEL, 
        #     padding_idx=config.TARGETS_MAPPING["[PAD]"]
        # )
        
        # token embedding
        self.token_embedding = nn.Linear(in_features=config.NUM_TOKENIZED_INPUTS, out_features=config.D_FF)
               
        # transformer layers
        self.layers = nn.ModuleList([
            TransformerDecoderLayer()
            for _ in range(num_layers)
        ])
        
        self._initialize()
    
    def _initialize(self):
        
        # Glorot / fan_avg. Initialization
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform(p)
            
    def forward(
        self, 
        inp:torch.Tensor, 
        mask:torch.Tensor=None, 
    ):
        
        B, T, D = inp.shape
        
        # positions = torch.eye(T).repeat(B, 1, 1)
        
        # embed tokens
        inp = self.token_embedding(inp)
        # inp = inp + self.position_embedding(positions) # positions to be added 
        
        # run attention self-attention modules
        attn_Ws = []
        for layer in self.layers:
            inp, attn_W = layer(inp, mask=mask)
            if attn_W is not None:
                attn_Ws.append(attn_W)
                
        return inp, attn_Ws

#### Test MultiHeadAttention Layer

In [70]:

# class SelfAttentionHead(nn.Module):
#     def __init__(self, d_model:int=config.D_MODEL):
#         super(SelfAttentionHead, self).__init__()
#         self.inf = 1e9
#         self.d_k = d_model
        
#         self.dropout = nn.Dropout(config.DROPOUT_RATE)
#         self._softmax = nn.Softmax(dim=-1)

#         self.w_q = nn.Linear(d_model, d_model, bias=False)
#         self.w_k = nn.Linear(d_model, d_model, bias=False)
#         self.w_v = nn.Linear(d_model, d_model, bias=False)
        
#         self.output_layer = nn.Linear(d_model, d_model)

#     def self_attention(
#         self, 
#         inp:toch.Tensor, 
#         mask=None, 
#         return_weights=True
#     ):
#         # Linear calculation
#         q = self.w_q(inp)
#         k = self.w_k(inp)
#         v = self.w_v(inp)

#         # Scaled Dot-Product Attention
#         matmul_qk = torch.matmul(q, k.transpose(-2, -1))
#         scaled_attention_logits = matmul_qk / math.sqrt(self.d_k)

#         if mask is not None:
#             scaled_attention_logits += (mask * -self.inf)

#         attention_weights = self._softmax(scaled_attention_logits)
#         attention_weights = self.dropout(attention_weights)

#         output = torch.matmul(attention_weights, v)

#         if return_weights:
#             return output, attention_weights
#         else:
#             return output, None

#     def forward(
#         self, 
#         inp:torch.Tensor, 
#         mask=None, 
#         return_weights=False
#     ):
        
#         attn_values, attn_W = self.self_attention(
#             inp, 
#             mask=mask, 
#             return_weights=return_weights
#         )
        
#         return attn_values, attn_W

# class MultiHeadSelfAttention(nn.Module):
#     def __init__(
#         self, 
#         d_model:int=config.D_MODEL, 
#         num_heads:int=config.N_HEADS
#     ):
#         super(MultiHeadSelfAttention, self).__init__()
        
#         self.num_heads = num_heads
#         self.head_dim = d_model // num_heads

#         self.attention_heads = nn.ModuleList([SelfAttentionHead(d_model) for _ in range(num_heads)])

#     def forward(
#         self, 
#         inp:torch.Tensor,  
#         mask=None, 
#         return_weights=True
#     ):
#         B = inp.shape[0]
        
#         head_outputs = [head(inp, mask=mask, return_weights=return_weights) for head in self.attention_heads]

#         # Combine the results from different heads
#         combined_output = torch.cat(
#             [output[0].view(B, -1, self.num_heads, self.head_dim) for output in head_outputs], 
#             dim=-1
#         )
#         attention_weights = torch.stack(
#             [output[1] for output in head_outputs], 
#             dim=1
#         )

#         return combined_output, attention_weights


In [72]:
mha = MultiHeadSelfAttention().cuda()
print(mha)
summary(mha)

MultiHeadSelfAttention(
  (attention_heads): ModuleList(
    (0-7): 8 x SelfAttentionHead(
      (dropout): Dropout(p=0.15, inplace=False)
      (_softmax): Softmax(dim=-1)
      (w_q): Linear(in_features=512, out_features=512, bias=False)
      (w_k): Linear(in_features=512, out_features=512, bias=False)
      (w_v): Linear(in_features=512, out_features=512, bias=False)
      (output_layer): Linear(in_features=512, out_features=512, bias=True)
    )
  )
)


Layer (type:depth-idx)                   Param #
MultiHeadSelfAttention                   --
├─ModuleList: 1-1                        --
│    └─SelfAttentionHead: 2-1            --
│    │    └─Dropout: 3-1                 --
│    │    └─Softmax: 3-2                 --
│    │    └─Linear: 3-3                  262,144
│    │    └─Linear: 3-4                  262,144
│    │    └─Linear: 3-5                  262,144
│    │    └─Linear: 3-6                  262,656
│    └─SelfAttentionHead: 2-2            --
│    │    └─Dropout: 3-7                 --
│    │    └─Softmax: 3-8                 --
│    │    └─Linear: 3-9                  262,144
│    │    └─Linear: 3-10                 262,144
│    │    └─Linear: 3-11                 262,144
│    │    └─Linear: 3-12                 262,656
│    └─SelfAttentionHead: 2-3            --
│    │    └─Dropout: 3-13                --
│    │    └─Softmax: 3-14                --
│    │    └─Linear: 3-15                 262,144
│    │    └─Linear: 3-16  

In [73]:
mem, attn_w = mha(inp=learned_tokens, return_weights=True)

mem.shape, attn_w.shape

(torch.Size([4, 48, 8, 512]), torch.Size([4, 8, 48, 48]))

In [ ]:
plot_self_attention(attn_w)
plot_self_attention(attn_w, example_idx=2)

#### Test Transformer decoder

In [257]:
dec = TransformerDecoder()
summary(model=dec)

Layer (type:depth-idx)                   Param #
TransformerDecoder                       --
├─Linear: 1-1                            25,088
├─ModuleList: 1-2                        --
│    └─TransformerDecoderLayer: 2-1      --
│    │    └─MultiHeadAttention: 3-1      1,049,088
│    │    └─LayerNormalization: 3-2      1,024
│    │    └─MultiHeadAttention: 3-3      1,049,088
│    │    └─LayerNormalization: 3-4      1,024
│    │    └─Sequential: 3-5              525,312
│    │    └─LayerNorm: 3-6               1,024
│    │    └─Dropout: 3-7                 --
│    └─TransformerDecoderLayer: 2-2      --
│    │    └─MultiHeadAttention: 3-8      1,049,088
│    │    └─LayerNormalization: 3-9      1,024
│    │    └─MultiHeadAttention: 3-10     1,049,088
│    │    └─LayerNormalization: 3-11     1,024
│    │    └─Sequential: 3-12             525,312
│    │    └─LayerNorm: 3-13              1,024
│    │    └─Dropout: 3-14                --
│    └─TransformerDecoderLayer: 2-3      --
│    │    └

In [ ]:
dec_out = dec(
    src=src_enc, 
    encoder_outputs=tokens,
    y=None #sample["motor_cmd"]["ids"].cuda()
)

In [ ]:
src_enc.shape

#### Action Generator

In [251]:
class ActionGenerator(nn.Module):
    def __init__(
        self,
        d_model:int=config.D_MODEL, 
        vocab_size:int=len(config.TARGETS),
        action_bins:int=config.ACTION_BINS,
        num_actons:int=config.NUM_ACTION_SLOTS
    ):
        super().__init__()
        
        # attrs
        self.action_bins = action_bins
        
        # layers
        self.norm = LayerNormalization()
        self.proj = nn.Linear(in_features=d_model, out_features=vocab_size)
        self._softmax = nn.LogSoftmax(dim=-1)

    def forward(self, tokens):
        
        out = self.norm(tokens)
        out = self.proj(out)
        out = self._softmax(out)
        
        return out
    

In [252]:
generator = ActionGenerator().cuda()
print(generator)
summary(generator)

ActionGenerator(
  (norm): LayerNormalization(
    (layer): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
  )
  (proj): Linear(in_features=512, out_features=52, bias=True)
  (_softmax): LogSoftmax(dim=-1)
)


Layer (type:depth-idx)                   Param #
ActionGenerator                          --
├─LayerNormalization: 1-1                --
│    └─LayerNorm: 2-1                    1,024
├─Linear: 1-2                            26,676
├─LogSoftmax: 1-3                        --
Total params: 27,700
Trainable params: 27,700
Non-trainable params: 0

In [ ]:
N, C, T = tokens.shape

N, C, T

In [ ]:
actions = generator(tokens.view(N, T, C))

actions.shape

### RT-1 Decoder

In [ ]:
class RT1Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.positional_encoder = PositionalEncoder()
        self.transformer = TransformerDecoder()
        self.action_generator = ActionGenerator()
        
    def _decode_predictions(self, preds, method:str="greedy"):
        """
            Args:
                preds: predictions (logits)
                method: decoding strategy. one of ["greedy", "beam-search"]
                
            Returns:
                actions: decoded predictions as sequence of actions.
            
        """
        pass
    
    def forward(self, learned_tokens):
        
        # positional encoding
        pos_enc = self.positional_encoder(imgs)
        
        # Generate attention mask
        attention_mask = generate_causal_attention_mask()
        # Add positional encoding
        pos_enc = pos_encoding(learned_tokens)
        # print(pos_enc.shape)
        learned_tokens = learned_tokens + pos_enc
        # Attend to learned tokens
        attended_tokens = self.transformer(learned_tokens, mask = ~attention_mask)
        out = reduce(attended_tokens, 'b (f n) d -> b f d', 'mean', f = (config.NUM_HISTORY + 1))
        
        # generate outputs
        logits = self.action_generator(out)
        
        return logits

### RT-1 

In [ ]:
class RT1(pl.LightningModule):
    def __init__(
        self
    ):
        super().__init__()
        self.encoder = RT1Encder()
        self.decoder = RT1Decoder()
        
    def forward(self, input_ids, attn_mask, token_type_ids, imgs):
        
        tokens = self.encode(input_ids, attn_mask, token_type_ids, imgs)
        out = self.decode(tokens, )
    
    def encode(self, input_ids, attn_mask, token_type_ids, imgs):
        return self.encoder(input_ids, attn_mask, token_type_ids, imgs)
    
    def decode(self, enc_outputs, x_mask, y, y_mask):
        return self.decoder(y, enc_outputs, src_mask, tgt_mask)
    
    def configure_optimizers(self):
        pass
    
    def training_step(self, batch, batch_idx):
        pass
    
    def validation_step(self, batch, batch_idx):
        pass
    
    def test_step(self, batch, batch_idx):
        pass
    
    def compute_loss(self, outputs, targets):
        pass